In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import pmdarima as pm
from pmdarima.utils import diff
from sklearn.metrics import mean_squared_error
from pmdarima.metrics import smape
import pickle
import sklearn
import statsmodels.tsa.statespace as sm

In [2]:
df = pd.read_csv('./../data/zri_acs_bikeshare_merged.csv')

In [3]:
df['zip'] = df['zip'].map(lambda x: str(x))

In [4]:
df['zip'] = df['zip'].map(lambda x: '0' + x if len(x)<5 else x)

In [5]:
df['datetime']=df['datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [6]:
df = df.set_index('datetime')

In [7]:
df

,zip,City,State,Metro,CountyName,zri,year,month,percent_white,percent_black,...,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
datetime,,,,,,,,,,,,,,,,,,,,,
2014-01-01,01013,Chicopee,MA,Springfield,Hampden County,928.0,2014,1,0.728315,0.020951,...,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
2014-02-01,01013,Chicopee,MA,Springfield,Hampden County,931.0,2014,2,0.727052,0.021179,...,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
2014-03-01,01013,Chicopee,MA,Springfield,Hampden County,934.0,2014,3,0.725788,0.021408,...,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
2014-04-01,01013,Chicopee,MA,Springfield,Hampden County,929.0,2014,4,0.724525,0.021636,...,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
2014-05-01,01013,Chicopee,MA,Springfield,Hampden County,929.0,2014,5,0.723261,0.021864,...,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1277.0,2019,8,0.796036,0.008872,...,60057.000000,0.102326,0.049199,33.566667,0.475244,0.399400,0.018076,0.0,0.0,0
2019-09-01,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1271.0,2019,9,0.795709,0.008765,...,60203.000000,0.102254,0.049419,33.575000,0.475511,0.399650,0.018306,0.0,0.0,0
2019-10-01,99654,Wasilla,AK,Anchorage,Matanuska Susitna Borough,1299.0,2019,10,0.795381,0.008658,...,60349.000000,0.102181,0.049640,33.583333,0.475778,0.399900,0.018537,0.0,0.0,0


In [8]:
zips = list(df['zip'].unique())

y = df.drop(columns = ['year', 'month', 'City', 'State', 'Metro', 'CountyName']).loc[:,['zip', 'zri']]
x = df.drop(columns = ['year', 'month', 'City', 'State', 'Metro', 'CountyName', 'zri'])

In [9]:
pred_dict = {}
coef_dict = {}

In [22]:
curr_x

,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,...,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
datetime,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.728315,0.020951,0.013732,0.219830,0.000000,0.004959,0.230010,0.311464,0.309663,0.148864,...,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
2014-02-01,0.727052,0.021179,0.013708,0.220867,0.000000,0.004917,0.229526,0.311552,0.310406,0.148516,...,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
2014-03-01,0.725788,0.021408,0.013685,0.221904,0.000000,0.004874,0.229041,0.311641,0.311150,0.148168,...,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
2014-04-01,0.724525,0.021636,0.013661,0.222940,0.000000,0.004832,0.228557,0.311729,0.311894,0.147820,...,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
2014-05-01,0.723261,0.021864,0.013638,0.223977,0.000000,0.004789,0.228072,0.311818,0.312638,0.147472,...,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,0.679939,0.027542,0.021597,0.259269,0.001507,0.001884,0.222496,0.340371,0.291326,0.145806,...,22998.333333,0.090141,0.028060,35.000000,0.517036,0.424700,0.044193,0.0,0.0,0
2019-09-01,0.679317,0.027160,0.021442,0.260334,0.001543,0.001955,0.222288,0.340476,0.291239,0.145998,...,23005.250000,0.089459,0.027907,35.000000,0.517142,0.425825,0.044099,0.0,0.0,0
2019-10-01,0.678695,0.026778,0.021287,0.261399,0.001579,0.002027,0.222079,0.340580,0.291151,0.146190,...,23012.166667,0.088777,0.027755,35.000000,0.517247,0.426950,0.044006,0.0,0.0,0


In [93]:
start = datetime.now()
for item in zips[0:1]:
    curr_y = y[y['zip']==item]
    curr_x = x[x['zip']==item].drop(columns = 'zip')
    curr_x_test = curr_x.iloc[60:,:]
    
    
    curr_model = sm.sarimax.SARIMAX(curr_y['zri'], exog = curr_x, order = (1,1,0), seasonal_order=(0, 0, 0, 0)).fit()
    
    pred_dict[item] = curr_model.predict(start = 1, exog = curr_x_test, dynamic = 59)
    coef_dict[item] = curr_model.params

elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.



ValueError: too many values to unpack (expected 2)

In [62]:
def plot_arima(zip_code, pred_dict):
    fig = go.Figure()
    fig.add_trace(go.Line(x = y[y['zip']==zip_code].index, y = y[y['zip']==zip_code]['zri'], mode = 'lines'))
    fig.add_trace(go.Line(x = pred_dict[zip_code].index, y = pred_dict[zip_code], mode = 'lines'))
    fig.add_vline(x=y[y['zip']==zip_code].index[60], line_width=3, line_dash="dash", line_color="green")
    fig.show()

In [63]:
plot_arima('10028', pred_dict)

In [64]:
pred_df = pd.DataFrame(pred_dict)

In [65]:
actual_dict = {}

In [66]:
for item in zips:
    curr_y = y[y['zip']==item]
    actual_dict[item] = curr_y.iloc[60:,1]

In [67]:
actual_df = pd.DataFrame(actual_dict)

In [68]:
resid_df = (pred_df.iloc[59:,:] - actual_df).T

In [69]:
resid_df

,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01
01013,11.633060,15.338032,10.544682,5.986130,7.537468,-0.859763,-7.232922,-13.594816,-16.951438,-33.305591,-15.658590,-6.011048
01020,18.380454,19.575646,12.712027,12.829739,16.941525,8.051429,-2.839264,-9.730146,-13.621088,-33.512050,-24.403018,-15.293988
01040,-4.450587,-18.023535,-21.728945,-19.110673,-14.371634,-21.587538,-26.786629,-33.979448,-37.169927,-51.359533,-22.548813,-8.737972
01085,-8.532262,-10.462644,-12.632110,-6.945155,-9.344423,-11.795472,-10.277616,-4.778435,-0.290468,13.190765,18.167953,23.142713
01104,1.403165,4.631132,2.173358,4.898454,10.729964,12.623397,13.552864,16.503298,14.465934,-2.564329,-5.590461,-3.614189
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,6.209487,2.135230,-5.115621,-16.476377,-20.905536,-19.377266,-14.875492,-5.390207,8.084815,33.553449,21.018109,8.480294
99501,24.914657,33.900916,42.266766,47.218023,42.892227,37.381286,31.746623,24.029280,12.256687,-23.552828,-30.387016,-37.237693
99504,16.942494,16.447663,9.689540,2.772901,-5.239174,-9.308705,-11.412828,-10.537777,-15.675265,-32.820302,-52.969884,-35.122202
99508,19.845321,21.077860,12.902484,5.455570,-6.172102,-16.920100,-24.748196,-34.629611,-42.546520,-55.487056,-59.443320,-61.410053


In [70]:
resid_df.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [71]:
np.sqrt(((resid_df**2).sum().sum())/(1301*12)) # RMSE for sarimax w/ bikeshare and ACS data

49.89143083787793

In [72]:
resid_df.to_csv('./sarimax_resids_acs_bikeshare.csv', index = True)

In [73]:
pd.DataFrame(coef_dict)

,01013,01020,01040,01085,01104,01108,01109,01420,01440,01453,...,98409,98444,98466,98498,98499,99207,99501,99504,99508,99654
percent_white,-279.009204,-799.667480,173.052726,83.039054,10.191712,85.187886,-130.429175,-548.454589,-135.929498,-488.293155,...,8.635786,-300.204032,-1023.795795,-608.632056,-271.119418,26.662997,-230.107442,-35.872355,181.858648,-371.655314
percent_black,-26.279856,-165.766820,-18.780075,-54.513667,158.715175,-274.387330,-449.019849,38.330955,-61.279023,-411.144219,...,-48.266582,-55.593724,52.229931,425.033809,395.392273,91.114326,-62.674898,5.936944,312.971619,435.583023
percent_asian,-46.514893,49.920378,41.810946,-46.891678,-176.244771,-12.615732,24.312008,183.131502,-359.885070,-109.364413,...,-7.194779,-110.543040,308.352663,-452.566172,163.189034,-31.358044,150.791661,3.691933,-673.824640,81.059736
percent_hispanic,320.986199,584.904031,-185.601530,10.388841,-50.339779,117.213523,486.909722,286.852146,537.335395,1158.770440,...,30.071443,220.459816,-188.802429,-520.530949,237.949067,-21.319400,117.484212,14.876855,-20.442504,57.571229
percent_native_am,8.023434,-16.174059,-8.550906,15.584634,2.875343,-11.337236,-3.875458,20.580439,8.302051,-60.950876,...,50.278912,5.668743,-69.043702,207.101540,-151.125047,-16.916612,23.747577,10.059326,-504.572658,-320.414224
percent_other_race,26.120137,-49.236368,-8.496106,11.218498,3.507352,29.523978,-0.451612,-16.965458,-3.711831,-45.093474,...,1.192380,37.146696,-43.945617,-22.348642,20.664256,0.667063,2.901464,-0.096621,-29.826361,0.694454
percent_0_17,-2.912550,-332.735858,-126.538868,-36.665893,177.689272,113.485564,-123.122627,142.504612,-12.898564,51.008628,...,28.036032,-21.326026,60.635298,-309.636880,292.993926,-88.284889,-66.063604,-38.942392,-406.594177,-169.152830
percent_18_39,-38.933997,696.178940,111.292281,84.123523,-105.050414,78.574664,130.675964,-24.139149,-155.989533,-50.860784,...,-13.985639,-42.023505,-163.376024,532.671866,-242.711094,20.297871,88.090978,2.969054,-170.866704,353.523041
percent_40_64,-3.587812,53.061440,-20.303229,-119.637555,-215.840056,-139.578323,-32.208828,-71.805770,361.014991,-41.489039,...,10.841309,38.099165,-243.257603,22.757221,156.559905,92.096062,-27.387755,82.085072,343.993889,-288.225050
percent_65+,45.433804,-416.503710,35.549817,72.184719,143.201078,-52.481906,24.654677,-46.569870,-192.123602,41.343416,...,-24.883393,25.253512,346.004205,-245.792206,-206.840846,-24.109043,5.362215,-46.111729,233.480230,103.859769


In [78]:
coef_df = pd.DataFrame(coef_dict)

In [79]:
coef_df = pd.DataFrame(coef_df.sum(axis=1)/1301) # average coefficients

In [81]:
coef_df.to_csv('./sarimax_coefs_acs_bikeshare.csv', index = True)

# Bikeshare data only

In [88]:
x_bike = pd.concat([x.loc[:,'zip'], x.loc[:, list(x.columns)[-3:]]], axis = 1)

In [89]:
x_bike

,zip,bs_total_stations,bs_total_systems,has_bike_sharing
datetime,,,,
2014-01-01,01013,0.0,0.0,0
2014-02-01,01013,0.0,0.0,0
2014-03-01,01013,0.0,0.0,0
2014-04-01,01013,0.0,0.0,0
2014-05-01,01013,0.0,0.0,0
...,...,...,...,...
2019-08-01,99654,0.0,0.0,0
2019-09-01,99654,0.0,0.0,0
2019-10-01,99654,0.0,0.0,0


In [94]:
pred_dict_bike = {}
coef_dict_bike = {}

In [95]:
start = datetime.now()
for item in zips:
    curr_y = y[y['zip']==item]
    curr_x = x_bike[x_bike['zip']==item].drop(columns = 'zip')
    curr_x_test = curr_x.iloc[60:,:]
    
    curr_model = sm.sarimax.SARIMAX(curr_y['zri'], exog = curr_x, order = (1,1,0), seasonal_order=(0, 0, 0, 0)).fit()
    
    pred_dict_bike[item] = curr_model.predict(start = 1, exog = curr_x_test, dynamic = 59)
    coef_dict_bike[item] = curr_model.params

elapsed = datetime.now()-start
print(elapsed)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

...

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning:

No frequency information was provided, so inferred frequency MS will be used.

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\statsmodels\tsa\b

0:03:45.295207


In [96]:
plot_arima('10028', pred_dict_bike)

C:\Users\fayma\anaconda3\envs\env1\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [97]:
pred_df_bike = pd.DataFrame(pred_dict_bike)

In [99]:
resid_df_bike = (pred_df_bike.iloc[59:,:] - actual_df).T

In [100]:
resid_df_bike

,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01,2019-10-01,2019-11-01,2019-12-01
01013,6.451602,5.353080,-3.887985,-12.621950,-15.056017,-27.312724,-37.464541,-47.554325,-54.607423,-74.638826,-60.657397,-54.668380
01020,13.000000,9.000000,-3.000000,-8.000000,-9.000000,-23.000000,-39.000000,-51.000000,-60.000000,-85.000000,-81.000000,-77.000000
01040,-5.180003,-19.612628,-24.218881,-22.475433,-18.584000,-26.629943,-32.649384,-40.657612,-44.661093,-59.662567,-31.663190,-18.663454
01085,-12.783971,-19.044608,-25.595065,-24.321736,-31.155549,-38.054504,-40.993068,-39.955713,-39.933001,-30.919192,-30.410796,-29.905691
01104,-2.811754,-8.785275,-19.878571,-25.352209,-27.389619,-33.119934,-39.634061,-43.995995,-53.250790,-77.430160,-87.556433,-92.645326
...,...,...,...,...,...,...,...,...,...,...,...,...
99207,8.674134,3.245544,-7.843919,-24.659126,-35.593588,-41.253956,-44.385121,-42.821626,-37.456165,-20.219140,-41.065414,-61.965713
99501,29.382460,42.003820,53.440298,61.079579,59.182499,55.924552,52.423811,46.759716,36.985716,3.137770,-1.759927,-6.691096
99504,23.700605,28.983018,27.390736,25.258971,21.794470,22.124748,24.328453,29.454092,28.531582,15.579375,-0.391148,21.627033
99508,27.027245,34.730361,32.544714,30.762520,24.579921,19.128567,16.496822,11.743997,8.909903,1.021261,2.096004,5.146173


In [101]:
resid_df_bike.columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [102]:
np.sqrt(((resid_df_bike**2).sum().sum())/(1301*12)) # RMSE for sarimax w/ bikeshare and ACS data

61.168600807209955

In [103]:
coef_df_bike = pd.DataFrame(coef_dict_bike)

In [104]:
coef_df_bike = pd.DataFrame(coef_df_bike.sum(axis=1)/1301) # average coefficients

In [105]:
coef_df_bike

,0
bs_total_stations,-0.056108
bs_total_systems,-1.298586
has_bike_sharing,-0.821477
ar.L1,0.611727
sigma2,170.049880
